# Python startup 

In [ ]:
def reverse(value):
    output = value[::-1]
    return output

string = reverse("hello")
print(string)

import numpy as np

dt = [1,2,3,4,5]

arr = np.zeros(5)
print(arr)

### Read csv file and write it to another csv or txt

In [ ]:
import csv

with open(r'C:\Users\pavan\OneDrive\Desktop\DataEng\Python\california_housing_test.csv', mode = 'r') as file:
    data = csv.reader(file)
    
    with open('output.csv', mode = 'w') as output_file:
        output_writer=csv.writer(output_file,delimiter='-')
        for rec in data:
            output_writer.writerow(rec)

   
file.close()
output_file.close()


### Pandas and its funtions 

In [ ]:
import pandas as pd

df = pd.read_csv('output.csv')

print("Raw data\n",df.to_string())

new_df = df.dropna()
print("altered data:\n",new_df.to_string())

new_df = df.fillna('filled')
print("new data\n", new_df)

df['Age'].fillna(100, inplace=True)
print("tranformed data\n",df)

avg_age = df['Age'].add_suffix('|')
print(avg_age.sort_values(ascending=True))

In [ ]:
def nth(data,n):
    
    df = pd.DataFrame(data,columns=['Id', 'Salary'])
    print("data:\n",df)
    print("length of data:",len(df))
    
    sorted_data = df['Salary'].sort_values(ascending=False)
    print(sorted_data)
    
    if len(sorted_data)<n:
        print("There is no {} highest salary".format(n))
    else:
        print('{} highest salary is:'.format(n),sorted_data.iloc[n])


data = [[12, 900], [2, 200], [3, 300],[9,300]]
result = nth(data,20)


In [ ]:
data = {'Department': ['Sales', 'Sales', 'HR', 'HR', 'Engineering', 'Engineering'],
 'Salary': [60000, 80000, 75000, 65000, 72000, 90000]}
df = pd.DataFrame(data)

df = df.groupby('Department')['Salary'].max().reset_index()

df = df[df['Salary']> 82000]

print(df)

In [ ]:
import psycopg2
import pandas as pd
import csv

    
cursor = None 
connection = None 
country=[]

try:
    connection = psycopg2.connect(database="postgres", user="postgres", password="root", host='192.168.5.56', port=5432)
    cursor = connection.cursor()

    #sql section 
    sel_script = ''' select * from country; '''
    cursor.execute(sel_script)
    rows = cursor.fetchall()
    
    #sql to Data Frame
    df = pd.DataFrame(rows)
    df = df.sort_values(by=0,ascending=True)
    limit = df.__len__()    
    
    #output section 
    with open("country.txt",mode = 'w') as of:
        of_writer=csv.writer(of)
        for i in range(0,limit):
            x=df.iloc[i]
            x= x.str.upper()
            of_writer.writerow(x)
            
    print("{} file created".format("country.txt"))
    
    #New table definition
    table_name = 'country_new'
    column1 = 'id'
    column2 = 'name'
    
    try:
        sql_create = '''create table if not exists 
                            {} (
                            {} INT,
                            {} VARCHAR(20)
                                );
                        truncate table {};'''.format(table_name,column1,column2,table_name)
                                
        cursor.execute(sql_create)
        connection.commit() 

        print("Table created")
    except Exception as error:
            print("table created failed:",error)
        

    try:
        id=0  
        for rec in range (0,limit-1):
            id+=1
            val=df[0][rec]
            val=val.upper()
            sql_insert = ''' insert into {} ({},{})
                                    values (%s,%s);
                            '''.format(table_name,column1,column2)
            
            cursor.execute(sql_insert,(id,val))
            connection.commit()
        
        print("Records inserted:",id)    
    except Exception as error:
        print("table insertion failed:",error)
        

except Exception as error:
    print(error)
finally:
    if cursor and connection is not None:
        of.close()
        cursor.close()
        connection.close()
        print("Program execution complete")    

### python and snowflake

In [ ]:
import pandas as pd
import snowflake.connector
import csv

con = snowflake.connector.connect(
    account='RGXQNUE-AW70262',
    user='SAI',
    password='Plmnko@5',
    warehouse='COMPUTE_WH',
    database='DBT_SNOWFLAKE',
    schema='DBT_SNOWFLAKE_SCHEMA'
    )

cur = con.cursor()

sql_script = "select * from chargeback"

cur.execute(sql_script)
rows = cur.fetchall()
df = pd.DataFrame(rows)

print(df)

### Python and PDF (cover)

In [ ]:
import re
from pypdf import PdfReader
import pandas as pd
import snowflake.connector
from datetime import datetime

cover = PdfReader(r'C:\Users\pavan\OneDrive\Desktop\DataEng\Python\Docs\Cover Letter.pdf')

# print('Folio letter total pages: ',len(folio.pages))
# print('Cover letter total pages: ',len(cover.pages))

cover_lines = cover.pages[0].extract_text()

cover_lines=cover_lines.splitlines()

#regex pattern
pattern_card = r'(\d{4})'
pattern_date = r'(\d{2}/\d{2}/\d{2})'
pattern_case = r'(CB\d)'
pattern_folio = "[0-9]"
pattern_amount = "[$]+[0-9]+[.]+[0-9]"
date_format = "%m/%d/%y"

lst,dates,cards,case_number,Address,amount,folio_no = [[] for _ in range(7)]

row = 0
Merchant_Address=''
for case in cover_lines:
    if case != ' ' and row <=2:
        Merchant_Address+=case
        row+=1
    
    lst = case.split(' ')
    
    for i in lst:
        
        #fetch card
        match = re.search(pattern_card,i)
        if match:
            i= i.strip(',').strip(' ').strip(').')
            if len(i) == 4:
                Credit_card=i 
        
        #fetch case
        i=i.upper()
        match = re.search(pattern_case,i)
        if match:
            Case_number=i
            
        #fetch folio
        i=i.upper()
        match = re.search(pattern_folio,i)
        if match and i.isdigit() and len(i)>5:
            Folio_no = i
        
            
        #fetch date
        match = re.search(pattern_date,i)
        if match:
            dates.append(i)
        
        #fetch amount
        i=i.upper()
        match = re.search(pattern_amount,i)
        if match and i.startswith('$'):
            Amount= i
 
Transaction_Date = datetime.strptime((dates[0][0:8]),date_format)
Checkin_date = datetime.strptime((dates[0][0:8]),date_format)   
Checkout_date = datetime.strptime((dates[2][0:8]),date_format)

print("case",Case_number)
print("transaction_Date", Transaction_Date)
print("checkin_date", Checkin_date)
print("checkout_date", Checkout_date)
print("card", Credit_card )
print("Address",Merchant_Address)
print("Amount",Amount)
print("folio no",Folio_no)


con = snowflake.connector.connect(
    account='RGXQNUE-AW70262',
    user='SAI',
    password='Plmnko@5',
    warehouse='COMPUTE_WH',
    database='DBT_SNOWFLAKE',
    schema='DBT_SNOWFLAKE_SCHEMA'
    )
    
try:
    cur = con.cursor()

    create_sql_script = ''' 
                            create table if not exists chargeback_Cover 
                                        ( 
                                            Case_no varchar(30), 
                                            Folio_no varchar(30),
                                            Transaction_Date varchar(30),
                                            Checkin_date varchar(30),
                                            Checkout_date varchar(30),
                                            Credit_card int,
                                            Merchant_Address varchar(1000),
                                            Amount varchar(10)
                                        )'''

    insert_sql_script = '''Insert into chargeback_cover 
                            values (?,?,?,?,?,?,?,?)'''
    
    Data = [Case_number,Folio_no,Transaction_Date,Checkin_date,Checkout_date,Credit_card,Merchant_Address,Amount]
    
    cur.execute(create_sql_script)
    cur.execute(insert_sql_script,Data)
    print("Table created and records inserted")
    
except Exception as error:
    print(error)
finally:
    cur.close()
    con.close()
    print("all connections closed")

### Python and pdf(Folio)

In [ ]:
#IMPORT Reruired modules 
import re
from pypdf import PdfReader
import pandas as pd
import pdfplumber

pattern = "[a-z-A-Z0-9]+@[a-zA-Z]"
name = ['Name']
column_names = ['Name', 'Address','Phone', 'Country', 'ID_Type','Folio_No','DateIn','Dateout','Transaction_Date','Amount']
amouts = []


folio = PdfReader(r"C:\Users\pavan\OneDrive\Desktop\DataEng\Python\Docs\Crystal Reports - rptFolio1.rpt.pdf")# to extract data
with pdfplumber.open(r"C:\Users\pavan\OneDrive\Desktop\DataEng\Python\Docs\Crystal Reports - rptFolio1.rpt.pdf") as f: #to extract data from tables
    tables = f.pages[0].extract_table()

#start

raw_text = folio.pages[0].extract_text()
lines = raw_text.splitlines()

for line in lines:
    
    words = line.split(' ')  
    
    for word in words:
        
        flag=''
        
        email = re.search(pattern,word)
        if email:
            print(word)
            email = word
            
        elif word in ['Date','Out','In', ]:
            if word == 'Out':
                date_out = words[2]
                print(date_out)
            elif word == 'In':
                date_in = words[2]
                country = words[5]
                print(date_in,'\n',country)
                
        elif word in ['Phone']:
            phone = words[2]
            print(phone)
            
        elif word in ['Driver']:
            id_type= words[5]+' '+words[6]
            print(id_type)
            
        elif word in ['Name'] and flag!='x':
            folio_no = words[5]
            Name = words[2]+' '+words[3]+' '+words[4]
            flag = 'x'
            print(folio_no,'\n',Name)
            
        elif word in ['Fee'] and flag!='y':
            Transaction_Date = words[0]
            flag = 'y'
            print(Transaction_Date)

for list in tables:
        for data in list:
            try: 
                x = float(data)
                amouts.append(x)
            except:
                continue
        
        